# 🧪 Тестовое задание для аналитика


Добро пожаловать! Это тестовое задание поможет нам оценить ваше знание Python (pandas), SQL, работы с базой данных, а также базовые навыки работы с Docker и сервером.

📁 Данные: в папке находится файл `orders_report.xlsx`, содержащий «грязные» данные из CRM.

Заполните ячейки ниже.


## 1. Обработка Excel-файла


🔹 **Задание:**
- СДЕЛАТЬ EDA АНАЛИЗ;
- ОБРАБОТАТЬ ДАННЫЕ, ПОЧИСТИТЬ И НЕ ПОТЕРЯТЬ ВАЖНЫЕ ДАННЫЕ

- ПОСЧИТАТЬ:
  - количество уникальных заказов;
  - топ-3 товара по выручке;
  - средний чек по каналу продаж.


✏️ **Ответ:**

In [ ]:
#ОТВЕТ
import pandas as pd

# Чтение файла
df = pd.read_excel(r"C:\Users\GEORGY\Downloads\orders_report_TZ.xlsx")  


#Убираем лишние строки и оформляем названия столбцов таблицы
df = df.iloc[2:] 
df.columns = df.iloc[0]
df = df.iloc[1:] 
df = df.reset_index(drop=True)
df.columns.name = None

#Удаляем строки у которых во всех столбцах NaN - считаем их "мусорными"
df = df.dropna(how='all')
df = df.reset_index(drop=True)

#Вывод статистик таблицы
print('Сводка по таблице:')
print(
    df.info(), 
    '\n'
)
print('Количество пропусков в каждом столбце:')
print(df.isnull().sum(),
      '\n')
print('Количество дубликатов в столбце')
print(df.duplicated().sum(),
      '\n')


#Делаем предобработку колонки для того, чтобы можно было ее в дальнейшем приаодить к числовым типам.
df['price_per_unit'] = df['price_per_unit'].astype(str)
df["price_per_unit"] = (
    df["price_per_unit"]
    .str.replace(r"[^\d.]", ".", regex=True)  # Удаляем всё, кроме цифр, точек и запятых
)
# Заменяем все точки, кроме последней, на пустую строку
df["price_per_unit"] = df["price_per_unit"].str.replace(
    r"\.(?=.*\.)",  # Ищем точку, за которой есть ещё хотя бы одна точка
    "",             # Заменяем на пустую строку
    regex=True
)


#Тут обрабаываем колонку quantity, чтобы восстановить NaN в этой колонке - Заменяю NaN на значение, которое получаю при делении итогового чека  на количество товаров.
def replaces_NaN_in_quantity(val: str, idx: str)->int:
    """
        Заменяет NaN значения в колонке 'quantity' вычисленным значением на основе 'total' и 'price_per_unit'.
        
        Параметры:
            val (str): Текущее значение в колонке 'quantity'
            idx (str): Индекс текущей строки в DataFrame
            
        Возвращает:
            int: Целое число - либо исходное значение, либо вычисленное
    """
    if not pd.isna(val):
            return int(val)
    return int(float(df.loc[idx, 'total']) // float(df.loc[idx, 'price_per_unit']))
df['quantity'] = [replaces_NaN_in_quantity(val, idx) for idx, val in zip(df.index, df['quantity'])]


#Тут обрабаываем колонку channel, чтобы восстановить NaN в этой колонке - Заменяю NaN на моду в колонке.
mode_channel = df['channel'].mode().values[0]
df['channel'] = df['channel'].apply(lambda x: mode_channel if pd.isna(x) else x )


#Пропуски c строками client_id и order_id я решил убрать, так как их немного и не знаю по каким правлам формируются идентификаторы.
df.dropna(subset=['client_id', 'order_id'], inplace=True)

# 1.Считаем уникальные значения в колонке 'order_id' == количество уникальных заказов
unique_order_id = df['order_id'].nunique()

# 2.Группируем выручку по товарам и получаем топ-3 товара по выручке;
df['product_name'] = df['product_name'].str.lower()
sales_by_product_name = df.groupby('product_name')['total'].sum().sort_values(ascending=False)

# 3. Группируем по каналу продаж и считаем среднее по чеку == получаем средний чек по каналу продаж
average_check_in_the_sales_channel = df.groupby('channel')['total'].mean()

print(f"""
# Анализ продаж

## 1. Количество уникальных заказов
🛒 **Уникальных заказов:** `{unique_order_id}`

## 2. Топ-3 товара по выручке
🏆 **Лучшие товары:**
1. `{sales_by_product_name.index[0]}`: {sales_by_product_name.iloc[0]:.2f} ₽
2. `{sales_by_product_name.index[1]}`: {sales_by_product_name.iloc[1]:.2f} ₽
3. `{sales_by_product_name.index[2]}`: {sales_by_product_name.iloc[2]:.2f} ₽

## 3. Средний чек по каналам продаж
📊 **Средний чек:**
{average_check_in_the_sales_channel.to_markdown()}
""")

Сводка по таблице:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        51 non-null     object
 1   client_id       51 non-null     object
 2   product_name    52 non-null     object
 3   channel         51 non-null     object
 4   quantity        51 non-null     object
 5   price_per_unit  52 non-null     object
 6   order_date      52 non-null     object
 7   total           52 non-null     object
dtypes: object(8)
memory usage: 3.4+ KB
None 

Количество пропусков в каждом столбце:
order_id          1
client_id         1
product_name      0
channel           1
quantity          1
price_per_unit    0
order_date        0
total             0
dtype: int64 

Количество дубликатов в столбце
0 


# Анализ продаж

## 1. Количество уникальных заказов
🛒 **Уникальных заказов:** `51`

## 2. Топ-3 товара по выручке
🏆 **Лучшие товары:**
1. `p

### 2. Загрузка данных в БД (**ОТРАЗИТЬ СИНТАКСИС И ШАГИ, К САМОЙ БД ПОДКЛЮЧАТЬСЯ НЕ НУЖНО!**)

### Задание:

    - Подключиться к БД 
        Пользователь - testuser
        Пароль - password  
        БД - testdb
        ХОСТ - localhost
        ПОРТ - 5432

    - СОЗДАТЬ таблицу "orders_cleaned" и загрузить датафрейм из 1-го задания
    - Завершить подключение


✏️ **Ответ:**

In [ ]:
#ОТВЕТ
import psycopg2
import pandas as pd
from psycopg2 import sql

# Параметры подключения к БД
db_params = {
    'host': 'localhost',
    'port': '5432',
    'database': 'testdb',
    'user': 'testuser',
    'password': 'password'
} 
# Подключаемся к БД
conn = psycopg2.connect(**db_params)
cursor = conn.cursor()

# Создаем таблицу orders_cleaned (предполагаем структуру)
create_table_query = """
DROP TABLE IF EXISTS orders_cleaned;
CREATE TABLE orders_cleaned (
    order_id INTEGER,
    client_id INTEGER,
    product_name VARCHAR(50),
    channel VARCHAR(255),
    quantity INTEGER,
    price_per_unit DOUBLE PRECISION,
    order_date TIMESTAMP,
    total DOUBLE PRECISION
);
"""
cursor.execute(create_table_query)
conn.commit()

for _, row in df.iterrows():
    try:
        # Формируем SQL-запрос с параметрами
        query = sql.SQL("""
            INSERT INTO orders_cleaned (
                order_id, client_id, product_name, 
                channel, quantity, price_per_unit, 
                order_date, total
            ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """)
        
        # Выполняем запрос с параметрами
        cursor.execute(query, (
            int(row['order_id']), 
            row['client_id'], 
            row['product_name'],
            row['channel'], 
            row['quantity'], 
            row['price_per_unit'],
            row['order_date'], 
            row['total']
        ))
        
    except Exception as e:
        print(f"Ошибка при вставке строки {_}: {e}")
        conn.rollback()  # Откатываем транзакцию при ошибке
        raise

conn.commit()
cursor.close()
conn.close()

## 3. SQL-запросы


Допустим, в БД появилась таблица `orders_cleaned`.

✍️ Напишите SQL-запросы:

1. Выручка по каждому товару за последний месяц.
2. Клиенты, сделавшие более 3 заказов.
3. Заказы, в которых одновременно есть товары X и Y.


In [ ]:
#ОТВЕТ
# ====================== ⚡ ИМПОРТ БИБЛИОТЕК ======================
import psycopg2
from datetime import datetime

# ====================== 🔐 ПАРАМЕТРЫ ПОДКЛЮЧЕНИЯ ======================
db_params = {
    'host': 'localhost',
    'port': '5432',
    'database': 'testdb',
    'user': 'testuser',
    'password': 'password'
} 

# ====================== 📊 ФУНКЦИЯ ДЛЯ КРАСИВОГО ВЫВОДА ======================
def print_results(title, rows, headers):
    print(f"\n{'='*50}")
    print(f"📌 {title.upper()}")
    print('='*50)
    
    if not rows:
        print("ℹ️ Нет данных для отображения")
        return
    
    # Определяем ширину колонок
    col_widths = [len(str(header)) for header in headers]
    for row in rows:
        for i, item in enumerate(row):
            if len(str(item)) > col_widths[i]:
                col_widths[i] = len(str(item))
    
    # Печатаем заголовки
    header = " | ".join(f"{h:<{col_widths[i]}}" for i, h in enumerate(headers))
    print(header)
    print("-"*len(header))
    
    # Печатаем данные
    for row in rows:
        print(" | ".join(f"{str(item):<{col_widths[i]}}" for i, item in enumerate(row)))

# ====================== 🚀 ОСНОВНОЙ КОД ======================
try:
    # Подключаемся к БД
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    
    print(f"\n🔗 Успешное подключение к БД {db_params['database']}")
    print(f"⏱ Время выполнения: {datetime.now().strftime('%d.%m.%Y %H:%M:%S')}\n")
    
    # ---------------------- 1. ВЫРУЧКА ПО ТОВАРАМ ----------------------
    cursor.execute("""
        SELECT product_name, sum(total) 
        FROM orders_cleaned 
        WHERE order_date >= NOW() - INTERVAL '1 month' 
        GROUP BY product_name
        ORDER BY sum(total) DESC
    """)
    print_results(
        "Выручка по товарам за последний месяц",
        cursor.fetchall(),
        ["Товар", "Сумма продаж"]
    )
    
    # ---------------------- 2. АКТИВНЫЕ КЛИЕНТЫ ----------------------
    cursor.execute("""
        SELECT client_id, count(client_id) 
        FROM orders_cleaned 
        GROUP BY client_id  
        HAVING count(client_id) >= 3
        ORDER BY count(client_id) DESC
    """)
    print_results(
        "Клиенты с 3+ заказами",
        cursor.fetchall(),
        ["ID клиента", "Кол-во заказов"]
    )
    
    # ---------------------- 3. КОМБИНИРОВАННЫЕ ЗАКАЗЫ ----------------------
    cursor.execute("""
        SELECT order_id 
        FROM orders_cleaned 
        WHERE product_name in ('X', 'Y') 
        GROUP BY order_id
        HAVING COUNT(DISTINCT product_name) = 2
    """)
    print_results(
        "Заказы с товарами X и Y",
        cursor.fetchall(),
        ["Номер заказа"]
    )

except Exception as e:
    print(f"\n❌ Ошибка: {e}")
finally:
    if 'conn' in locals():
        conn.close()
        print("\n🔌 Соединение с БД закрыто")
    print("\n✨ Анализ завершен")

## 4. Docker и серверные команды


✏️ Ответьте на следующие вопросы:

1. Как упаковать этот скрипт в Docker-контейнер?
2. Какие команды используете для:
   - перезапуска контейнера
   - просмотра логов
   - остановки зависшего процесса
3. Как бы вы настроили запуск скрипта каждый день в 8:00 через cron?
4. Что делать, если команда на сервере "зависла"? Как дебажить?


In [ ]:
# ОТВЕТ 
#1.Нужно сложить файл с зависимостями, скрипт и докер файл, в котором прописать копирование зависимостей и запуск скрипт при старт,
# затем нужно сохранить докер файл, затем командой docker build  собрать образ и запустить котнейнер.
#2.Docker restart контейнер, docker logs контейнер, docker stop контейнер.
#3. Делаем Докер файл, копируем крон задание и скрипт запуска крона, в файле крон задания прописываем необходимые условия на запуск каждый день  в 8
# (0 8 * * * root /usr/local/bin/python /app/script.py), указываем, что скрипт запуска будет запускаться при старте контейнера. Далее упаковываем образ и стартуем контейнер.
#4. Если команда на сервере зависла- надо проверить состояние процесса, посмотреть нагрузку на систему, далее можно посмотреть дерево процесса и потоков,
#  посмотреть системные и библиотечные вызовы процесса. Потом можно предпринять следующие действия - сделать мягкое завершение, если не помогло, то принудительное и перезапустить сервер.
# Возможно, стоит оценить лимиты и посмотреть, кто сколько ресурсов кушает, может потребуется некая оптимизация, изменение кода.

---
📨 Спасибо за выполнение задания!